<a name='T'></a>


#### Overview

## ZZY的毕设。用于一个专注于预测特斯拉(Tesla)股价的综合项目。所采用的预测模型包括用于即时预测的单步LSTM和用于扩展预测的多层LSTM，扩展预测跨越未来的多个天(相当于一个月的工作日)。


<a name='I'>
    
# <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -20px;">1| 引言 </p>
    
我们将探索长短期记忆(LSTM)网络的应用，这是一种特殊的递归神经网络(RNN)，用于预测特斯拉的股价。

lstm，即长短期记忆网络，专门用于识别序列数据中的复杂模式。这使得它们在涉及时间序列数据的任务中特别有效，比如预测股票价格，或者处理语言序列，比如短语和句子。与标准的前馈神经网络不同，lstm具有反馈连接，使它们不仅可以处理单个数据点，还可以处理整个数据序列。对于股票价格预测模型，这允许LSTM在预测未来价格时考虑股票价格的历史顺序。

### <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">1.1.1| LSTM的直观解释 </p>
    
LSTM单元被设计用来跟踪输入序列中的依赖关系。它通过一种叫做门的结构来调节信息的流动。这些门根据当前的输入和之前的状态决定哪些应该保留在记忆中(细胞状态)，哪些应该忘记。

- **忘记门$f_t$**:这个门决定从cell状态中哪些信息应该被丢弃或保留。它查看前面的输出$h_{t-1}$和当前的输入$x_t$，并对单元格状态$C_{t-1}$中的每个数字应用一个sigmoid函数来输出0到1之间的数字。1代表“完全保留这个”，而0代表“完全摆脱这个”。

- **输入门$i_t$**:此门用新信息更新单元状态。它首先使用sigmoid函数决定更新哪些值，然后创建一个新的候选值向量$\nu(C_t)$，它可以添加到状态中。

—**细胞状态$C_t$**:表示LSTM的内存。它是通过忘记被认为不必要的东西，并根据其重要性添加新的候选值来更新的。

—**输出门$o_t$**:最后计算输出。单元格状态通过“tanh”传递(将值推到-1和1之间)，然后乘以s形门的输出，因此我们只输出我们决定[1,2]的部分。

<center>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png"  width="60%" height="30%">
</center>

### <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">1.1.2| LSTM数学解释 </p>
    
LSTM单元在序列的每一步t上计算以下函数:

- **忘记门$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$ **:该门通过查看之前的输出和当前的输入来决定哪些信息与细胞状态无关。

- **输入门$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$ **:它决定哪些值将在单元状态更新。 

- **细胞输入$\tilde{C_t} = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$ **:它创建了一个新的候选值的向量，可以添加到细胞状态。

- **细胞状态更新$C_t = f_t * C_{t-1} + i_t * \tilde{C}_t$ **:细胞状态更新通过忘记的东西(乘以$f_t$)和添加新的候选值(乘以$i_t$)。

- **输出门$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$ **:它决定下一个隐藏状态($h_t$)应该是什么。

—**隐藏状态输出$h_t = o_t * \tanh(C_t)$ **:根据细胞状态输出，但只允许部分输出(由$o_t$控制)。 

这些步骤中的每一步都涉及权重($W$)、偏差($b$)以及在学习过程中训练的前一个隐藏状态和当前输入的组合。LSTM维持长期记忆的能力主要是由于细胞状态$C_t$以及门如何相互作用来修改这种记忆。

我们将从检查特斯拉股票数据集开始，该数据集包括每日交易信息，如开盘价、最高价、最低价、收盘价和交易量[1,2]。
    
<a name='D'>
    
# <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -20px;">2| 特斯拉股票数据集概述 </p>
    
我们将深入研究特斯拉公司(Tesla, Inc.)的股票数据。特斯拉以其在电动汽车、能源储存和太阳能电池板制造方面的创新方法而闻名。作为一家走在可持续能源转型前沿的公司，特斯拉的股票市场表现引起了投资者、市场分析师以及技术和可持续发展爱好者的极大兴趣。该数据集包括以几个关键财务指标捕获的每日交易信息:

-“open”:股票在某一天市场开盘时开始交易的价格。
-“high”:当天股票交易的最高价格。
- 'low ':股票当天交易的最低价格。
-“close”:股票在当天交易的最后价格。这是金融新闻中最常报道的数字。
-“volume”:在给定时期内，一种证券或整个市场中交易的股票或合约的数量。它是衡量库存总需求和总供给的指标。
-“dividends”:从公司收益的一部分中分配奖励，并支付给某一类股东。
-“stock splits”:公司将其现有股票分割成多股，以提高股票流动性的行为。尽管已发行股票的数量增加了一个特定的倍数，但与分拆前的数量相比，股票的总美元价值保持不变，因为分拆没有增加任何实际价值。

特斯拉的历史股票数据提供了对该公司股票长期表现的洞察。通过分析这些数据中的模式，我们可以尝试使用长短期记忆(LSTM)网络等机器学习技术来预测未来的股票价格。

<a name='is'>
    
# <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -20px;">3| 导入和设置</p>

In [1]:
!pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/14/03/3c854ca3e02eedf614abba4b2e177c469bf3af58207fa30d5098c5d652fe/yfinance-0.2.37-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 6.1 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.1-cp310-cp310-linux_x86_64.whl size=291650 sha256=2f967f54f51caf5d2695363f757fa425fdffdffc9b243d8b90da2d68510422c7
  Stored in directory: /root/.cache/pip/wheels/d7/35/5c/1374782be033462df5f40174d8d879519d64ed8c25a1977554
Succ

In [2]:
# Data Imports
import yfinance as yf
import pandas as pd 
import numpy as np 
import random

from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.offsets import BDay

# Visualization Imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import scipy.stats as stats

# Neural Network Imports
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint

# Setting seed
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Visualization Configurations
pio.templates.default = "plotly_dark"
%config InlineBackend.figure_format = 'retina'

<a name='tsladata'>
    
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">3.1| 特斯拉数据 </p>

In [3]:
def show(data: pd.DataFrame):
    df = data.copy()
    df = df.style.format(precision=3)
    df = df.background_gradient(cmap='Reds', axis=0)
    display(df)

def highlight_half(data: pd.DataFrame, axis=1, precision=3):
    
    s = data.shape[1] if axis else data.shape[0]
    data_style = data.style.format(precision=precision)

    def apply_style(val):
        style1 = 'background-color: red; color: white'
        style2 = 'background-color: blue; color: white'
        return [style1 if x < s//2 else style2 for x in range(s)]

    display(data_style.apply(apply_style, axis=axis))

In [4]:
# Target stock & columns for modeling
SYMBOL = "TSLA"
columns = ['open', 'high', 'low', 'close', 'volume']

# Getting Tesla (TSLA) stock data
ticker = yf.Ticker(SYMBOL)

# End stock dates
end_date = "2023-12-01"

# Pulling stock data 
df = ticker.history(start="2017-01-01", end=end_date)
df.columns = df.columns.str.lower()

# Showing data
show(df.tail())

,open,high,low,close,volume,dividends,stock splits
Date,,,,,,,
2023-11-24 00:00:00-05:00,233.750,238.750,232.330,235.450,65125200,0.000,0.000
2023-11-27 00:00:00-05:00,236.890,238.330,232.100,236.080,112031800,0.000,0.000
2023-11-28 00:00:00-05:00,236.680,247.000,234.010,246.720,148549900,0.000,0.000
2023-11-29 00:00:00-05:00,249.210,252.750,242.760,244.140,135401300,0.000,0.000
2023-11-30 00:00:00-05:00,245.140,245.220,236.910,240.080,132353200,0.000,0.000


In [5]:
# Data info
print('Data Info:')
df.info()

Data Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1740 entries, 2017-01-03 00:00:00-05:00 to 2023-11-30 00:00:00-05:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          1740 non-null   float64
 1   high          1740 non-null   float64
 2   low           1740 non-null   float64
 3   close         1740 non-null   float64
 4   volume        1740 non-null   int64  
 5   dividends     1740 non-null   float64
 6   stock splits  1740 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 108.8 KB


<a name='DV'>
    
# <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -65px;">4| 数据可视化 </p>
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">4.1| Candle Stick </p>
    

烛台图是一种用于市场分析的可视化工具，用于描述证券、衍生品或货币的价格走势。每个“烛台”通常代表一天的交易，由主体和灯芯组成。
- **主体**:烛台较宽的部分，表示开盘价和收盘价。如果身体被填满或黑暗，安全关闭低于它打开。如果身体是空的或轻的，它关闭的比打开的高。
- **线**:从主体的顶部和底部延伸的线，代表在此期间的最高和最低价格。


In [6]:
def plot_candlestick(stock_df, name='', rolling_avg=None, fig_size=(1100, 700)):
    """
    Plot a candlestick chart for the given stock dataframe with optional rolling averages.
    
    Args:
        stock_df (pd.DataFrame): The stock data as a pandas DataFrame.
        name (str): The name of the stock, defaults to 'Tesla'.
        rolling_avg (list of int, optional): A list of integers for rolling average window sizes.
        fig_size (tuple): The figure size, defaults to (1100, 700).
    """
    # Copy df to avoid modifying the original data
    stock_data = stock_df.copy()
    
    # Creating plot
    fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
        close=stock_data['close'], open=stock_data['open'], high=stock_data['high'], low=stock_data['low'], 
        name="Candlesticks", increasing_line_color='green', decreasing_line_color='red', line=dict(width=1)
                                        )])
    # Rolling averages if specified
    if rolling_avg:
        colors = ['rgba(0, 255, 255, 0.5)',   # cyan
                  'rgba(255, 255, 0, 0.5)',   # yellow
                  'rgba(255, 165, 0, 0.5)',   # orange
                  'rgba(255, 105, 180, 0.5)', # pink
                  'rgba(165, 42, 42, 0.5)',   # brown
                  'rgba(128, 128, 128, 0.5)', # gray
                  'rgba(128, 128, 0, 0.5)',   # olive
                  'rgba(0, 0, 255, 0.5)']     # blue
        
        for i, avg in enumerate(rolling_avg):
            color = colors[i % len(colors)]
            ma_column = f'{avg}-day MA'
            stock_data[ma_column] = stock_data['close'].rolling(window=avg).mean()

            # Moving average trace
            fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data[ma_column],
                    mode='lines', name=f'{avg}-day Moving Average', line=dict(color=color)))

    # Layout updates
    fig.update_layout(title=f"特斯拉股票-蜡烛图",
                      xaxis_title="Date", yaxis_title="Price",
                      width=fig_size[0], height=fig_size[1],
                      xaxis=dict(
                          rangeselector=dict(
                              buttons=list([
                                  dict(count=14, label="2w", step="day", stepmode="backward"),
                                  dict(count=1, label="1m", step="month", stepmode="backward"),
                                  dict(count=3, label="3m", step="month", stepmode="backward"),
                                  dict(count=6, label="6m", step="month", stepmode="backward"),
                                  dict(count=1, label="YTD", step="year", stepmode="todate"),
                                  dict(count=1, label="1y", step="year", stepmode="backward"),
                                  dict(count=2, label="2y", step="year", stepmode="backward"),
                                  dict(count=3, label="3y", step="year", stepmode="backward"),
                                  dict(count=5, label="5y", step="year", stepmode="backward"),
                                  dict(step="all")]),
                              bgcolor='pink',
                              font=dict(color='black'),
                              activecolor='lightgreen'))
                      
                     )
    
#     fig.update_layout(title=f"特斯拉股票-蜡烛图",
#                       xaxis_title="Date", yaxis_title="Price",
#                       width=fig_size[0], height=fig_size[1],
#                       plot_bgcolor='white')  # 设置图表的背景颜色为白色    
    fig.show()


In [7]:
# General Tesla stocks
plot_candlestick(df, name=SYMBOL)

# With Moving averages
plot_candlestick(df, name=SYMBOL, rolling_avg=[20, 50, 200])

<a name='splits'>
    
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">4.2| Tesla Stock Splits  </p>

In [8]:
# Plotting stock splits
fig = px.line(x=df.index, y=df['stock splits'], title=f'{SYMBOL} Stock Splits Over Time')
fig.update_layout(width=1100, height=500)
fig.update_traces(line=dict(color='cyan', width=3))
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Stock Splits')
fig.show()

<a name='pchange'> 

## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -20px;">4.3| Percent Change in Stock Prices </p>
股票价格的百分比变化是一种用来表示价格随时间变化与前一价格的比例的度量。这个指标对于比较一只股票在不同时间范围内的表现或与其他股票的表现很有用。计算每日百分比变化的公式为:
$$\text{Percentage Change} = \left( \frac{\text{Current Price} - \text{Previous Price}}{\text{Previous Price}} \right) \times 100
$$
- **当前价格**是股票在当期结束时的价格(例如，当天结束时)。
- **上期价格**是股票在上期结束时的价格。

对于时间序列数据集的第一个时期(例如，可用股票数据的第一天)，由于没有以前的价格可以比较，因此没有定义百分比变化。在这种情况下，通常将百分比更改设置为零或省略该值。在分析较长时期(如一个月或一年)的股票数据时，百分比变化是使用该时期开始和结束时的价格来计算的。这一指标通常用于财务分析，以评估波动性和股票的表现。它也是投资者决定买入或卖出证券的关键指标。

In [9]:
# Creating subplot
fig = make_subplots(rows=2, cols=2, column_widths=[0.7, 0.3],
                    vertical_spacing=0.1, horizontal_spacing=0.05,
                    subplot_titles=(f"{SYMBOL}-随时间的百分比变化", f"{SYMBOL} 百分比变化-直方图",
                                    f"{SYMBOL}-随时间变化的交易量", f"{SYMBOL} 股票交易量-直方图",))
# Percent Change Plot
percent_change = df['close'].pct_change() * 100
fig.add_trace(go.Scatter(x=df.index, y=percent_change, name='Percent Change', marker_color='darkorchid'), row=1, col=1)
fig.add_trace(go.Histogram(x=percent_change, nbinsx=50, name='Percent Change', marker_color='darkorchid'),  row=1, col=2)
fig.add_annotation(text=f"Mean: {percent_change.mean():.2f}%<br>Std Dev: {percent_change.std():.2f}%",
                   xref='x2', yref='y2', x=percent_change.mean(), y=5, showarrow=True)
# Volume Plot
fig.add_trace(go.Scatter(x=df.index, y=df['volume'], name='Volume', marker_color='darkcyan'), row=2, col=1)
fig.add_trace(go.Histogram(x=df['volume'], nbinsx=50, name='Daily Volume', marker_color='darkcyan'),  row=2, col=2)
fig.add_annotation(text=f"Mean: {df['volume'].mean():.2f}<br>Std Dev: {df['volume'].std():.2f}",
                   xref='x4', yref='y4', x=df['volume'].mean(), y=5, showarrow=True)

fig.update_layout(height=700, width=1100)
fig.show()

- **特斯拉随时间的百分比变化:**图显示了特斯拉股价的每日百分比变化，揭示了在观察期间的波动性。波动通过急剧的尖峰和低谷来捕捉，表明有重大价格变动的日子。这可能反映了市场对新闻事件、收益报告或更广泛的经济状况的反应。
​
- **特斯拉百分比变化-直方图:右上方的**图表示这些每日百分比变化的分布。大多数变化都集中在平均值附近，表明收益呈正态分布，这是股票价格随时间变化的典型特征。接近于零的平均值表示稳定的平均增长，而标准差表示偏离平均值的程度。
​
- **特斯拉股票交易量:**显示特斯拉股票的交易量。该图的峰值可能对应于影响投资者情绪和交易行为的特定事件或重大新闻的发布。
​
- **特斯拉股票交易量-直方图:**说明了交易量的分布，表明某些交易量发生的频率。平均值和标准偏差提供了典型体积及其可变性的摘要。数据集中在低端表明，高交易量的日子不那么频繁，但可能与关键市场或公司特定事件有关。

<a name='lstm'>
    
# <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -65px;">5| LSTM </p>
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -30px;">5.1| LSTM Data Prep </p>

在机器学习中，在将数据输入模型之前对其进行归一化通常是至关重要的，尤其是在使用神经网络时。这个过程将在不同尺度上测量的值调整到一个概念上的共同尺度，通常在平均之前。这里，我们使用来自“sklearn”的“MinMaxScaler”。预处理包，通过其最大值和最小值缩放每个特征。该标量使用以下公式将特征列中的每个值' v'转换为[0,1]范围内的' v' ':
$$v' = \frac{v - \text{min}(v)}{\text{max}(v) - \text{min}(v)}$$
—“v”为初始值。
- ' min(v) '为特征列中的最小值。
—“max(v)”为特征列中的最大值。
来自特斯拉股票数据集的列' ' '开盘价'，'高点'，'低点'，'收盘价'，'成交量']'被归一化，其中包括开盘价，高点，低点和收盘价以及交易量。在神经网络的训练过程中，将这些特征归一化可以使神经网络更稳定、更快地收敛，就像我们将用于股票价格预测的LSTM模型一样。

In [10]:
class RNNFormater:
    
    def __init__(self, data: pd.DataFrame, mapping_steps=10):
        """
        Initialize the RNNFormater with a DataFrame and steps to map for data.
        
        Args:
            data (pd.DataFrame): Input DataFrame containing time series data.
            mapping_steps (int): Number of time steps for each input sequence to be mapped to output.
        """
        # Storing data
        self.df = data.copy()
        self.data = self.df.values

        # Scaler stored for usage later
        self.scaler = MinMaxScaler()
        self.normalized_data = self.scaler.fit_transform(self.data)
        
        self.time_steps = data.shape[0]
        self.n_columns = data.shape[1]

        # Number of mapping steps
        self.mapping_steps = mapping_steps

    def data_mapping(self):
        """
        Maps a 2D array into a 3D array for RNNs input, with each sequence having mapping_steps time steps.
    
        Args:
            mapping_steps (int): Number of time steps for each sequence.
    
        Returns:
            np.array: A 3D array suitable for RNN inputs.
        """
        mapping_steps = self.mapping_steps + 1
        
        mapping_iterations = self.time_steps - mapping_steps + 1
        self.normalized_data_mapped = np.empty((mapping_iterations, mapping_steps, self.n_columns))
        
        for i in range(mapping_iterations):
            self.normalized_data_mapped[i, :, :] = self.normalized_data[i:i + mapping_steps, :]
        
        return self.normalized_data_mapped
    
    def rnn_train_test_split(self, test_percent=0.1):
        """
        Splits the 3D mapped data into training and testing sets for an RNN.
        
        Args:
            test_percent (float): The fraction of data to be used for testing.
        
        Returns:
            tuple: X_train, X_test, y_train, y_test
        """
        self.test_size = int(np.round(self.normalized_data_mapped.shape[0] * test_percent))
        self.train_size = self.normalized_data_mapped.shape[0] - self.test_size
        
        X_train = self.normalized_data_mapped[:self.train_size, :-1, :]
        y_train = self.normalized_data_mapped[:self.train_size, -1, :]
        
        X_test = self.normalized_data_mapped[self.train_size:, :-1, :]
        y_test = self.normalized_data_mapped[self.train_size:, -1, :]
        
        return X_train, X_test, y_train, y_test  

    def forecast_n_steps(self, model, data: pd.DataFrame, n_forecast_steps=30):
        """
        Forecast multiple steps ahead using the LSTM model.
    
        Args:
            model (tf.keras.Model): Trained LSTM model for prediction.
            data (pd.DataFrame): Input DataFrame containing the latest time series data.
            n_forecast_steps (int): Number of future steps to forecast.
    
        Returns:
            np.array: Forecasted values for n_forecast_steps.
        """
        # Scaling the latest 'mapping_steps' data for mapping
        last_steps = self.scaler.transform(data.values)[-self.mapping_steps:]
    
        # Initialize normalized_data_mapped array
        normalized_data_mapped = np.empty((n_forecast_steps, self.mapping_steps, self.n_columns))
    
        # Initialize predictions array
        predictions = np.empty((n_forecast_steps, self.n_columns))
    
        # Predict the first step
        normalized_data_mapped[0, :, :] = last_steps
        predictions[0, :] = model.predict(
            normalized_data_mapped[0, :, :].reshape(1, self.mapping_steps, self.n_columns),
            verbose=False
        )
        # Generate predictions and update normalized_data_mapped for each subsequent step
        for i in range(1, n_forecast_steps):
            # Shift the window and insert new prediction at end
            normalized_data_mapped[i, :-1, :] = normalized_data_mapped[i - 1, 1:, :]
            normalized_data_mapped[i, -1, :] = predictions[i - 1, :]
    
            # Predicting next step
            norm_data = normalized_data_mapped[i, :, :].reshape(1, self.mapping_steps, self.n_columns)
            predictions[i, :] = model.predict(norm_data, verbose=False)
    
        # Inverse transform the predictions to original scale
        predictions = self.scaler.inverse_transform(predictions)
        return predictions


In [11]:
# Initializing class
mapping_steps = 32 # ~ 1 months in buisness days
rnn_formater = RNNFormater(df[columns], mapping_steps=mapping_steps)

# Mapping steps
norm_data_mapped = rnn_formater.data_mapping() # n_steps -> y
# print(f'Mapped Normalized data step 0:\n{norm_data_mapped[0]}')
print(f'Normalized data shape: {norm_data_mapped[0].shape}')

Normalized data shape: (33, 5)


In [12]:
# Train Test Split
X_train, X_test, y_train, y_test = rnn_formater.rnn_train_test_split(test_percent=0.05)
print(f'Number of time steps for test set: {rnn_formater.test_size}')

print(f'X shape: {X_train.shape}')
# print(X_train[0])

print(f'y shape: {y_train.shape}')
# print(y_train[0])

Number of time steps for test set: 85
X shape: (1623, 32, 5)
y shape: (1623, 5)


<a name='buildmodel'>

## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">5.2| Vanilla LSTM Model Building </p>

In [13]:
# For consistant results
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# Vanilla LSTM
model = models.Sequential([
    LSTM(units=80, input_shape=(mapping_steps, len(columns))),
    Dropout(0.05),
    Dense(units=len(columns))   
])

# Compiling model
model.compile(optimizer='adam', loss='mae')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 80)                27520     
                                                                 
 dropout (Dropout)           (None, 80)                0         
                                                                 
 dense (Dense)               (None, 5)                 405       
                                                                 
Total params: 27925 (109.08 KB)
Trainable params: 27925 (109.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Callback to save model weights
model_checkpoint = ModelCheckpoint('LSTM_Tesla_model.h5', monitor='val_loss', save_best_only=True)

# Fitting the model
history = model.fit(X_train, y_train, 
                    batch_size=256, 
                    epochs=1_000, 
                    validation_data=(X_test, y_test), 
                    callbacks=[model_checkpoint],
                    shuffle=False,
                    verbose=False)

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [15]:
def plot_training_history(history, plot_title='Training Performance', plot_legends=None, color0=0):
    """
    Plots the training history of a model using Plotly.

    Args:
        history (dict): A dictionary containing the training history metrics.
        plot_title (str): Title of the plot.
        plot_legends (list): List of legends for the plot. If None, it uses the keys from the history dictionary.

    Returns:
        None: Displays the plot.
    """
    # Extracting metrics from the history object
    epochs = np.arange(1, len(next(iter(history.values()))) + 1)
    colors = ['blue', 'gold', 'violet', 'lime', 'blue', 'pink', 'yellow']
    data = []

    # If no legends are provided, use keys from the history
    if not plot_legends:
        plot_legends = list(history.keys())

    # Prepare data for each metric in the history
    for i, (key, legend) in enumerate(zip(history.keys(), plot_legends)):
        color_index = i % len(colors) + color0
        data.append(go.Scatter(x=epochs, y=history[key], mode='lines+markers', name=legend, line=dict(color=colors[color_index])))

    # Add error for minimum epoch value
    min_epoch = np.argmin(history['val_loss']) + 1 
    loss_str = f"Train Loss: {history['loss'][min_epoch-1]:.3e}<br>Test Loss: {history['val_loss'][min_epoch - 1]:.3e}"

    # Creating the layout
    layout = go.Layout(title=plot_title, xaxis=dict(title='Epochs'), yaxis=dict(title='Value'), width=1100, height=600)
    fig = go.Figure(data=data, layout=layout)

    # Annotate the minimum loss with an arrow
    fig.add_annotation(
        go.layout.Annotation(
            x=min_epoch,
            y=history['loss'][min_epoch - 1],
            xref="x",
            yref="y",
            text=loss_str,
            showarrow=True,
            arrowhead=7,
            arrowcolor='green',
            arrowsize=2,
            bordercolor='green',
            borderwidth=2,
            ax=0,
            ay=-40
        )
    )
    fig.show()

In [16]:
# Plotting LSTM model loss
plot_training_history(history.history, plot_title='LSTM Model Loss')

In [17]:
# Loading best wieghts during training
model = models.load_model(f'LSTM_Tesla_model.h5')

# Predicting
predictions = model.predict(X_test, verbose=False)
predictions = rnn_formater.scaler.inverse_transform(predictions)

# Showing predictions and data
index_1 = y_test.shape[0]
df_y_test = df[columns].iloc[-index_1:]
df_predictions = pd.DataFrame(predictions, index=df_y_test.index, columns=[f'pred_{col}' for col in columns])
df_test_pred = pd.concat([df_y_test, df_predictions], axis=1)
                         
# Shwoing outputs
highlight_half(df_test_pred.tail(), axis=1)

,open,high,low,close,volume,pred_open,pred_high,pred_low,pred_close,pred_volume
Date,,,,,,,,,,
2023-11-24 00:00:00-05:00,233.750,238.750,232.330,235.450,65125200,234.986,239.409,230.042,236.064,122568344.000
2023-11-27 00:00:00-05:00,236.890,238.330,232.100,236.080,112031800,235.859,240.129,231.261,237.284,97312840.000
2023-11-28 00:00:00-05:00,236.680,247.000,234.010,246.720,148549900,235.569,240.001,230.728,236.533,113297632.000
2023-11-29 00:00:00-05:00,249.210,252.750,242.760,244.140,135401300,243.920,248.929,238.931,244.756,132169640.000
2023-11-30 00:00:00-05:00,245.140,245.220,236.910,240.080,132353200,245.658,250.406,240.475,246.453,128477992.000


### <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red;">5.2.1| LSTM Resdiual Analysis Building </p>

In [18]:
# Error dataframe
df_error = pd.DataFrame(df_predictions.values - df_y_test.values, index=df.index[-index_1:], columns=[f'error_{col}' for col in columns])
print('RMSE Per Column')
print((df_error**2).mean()**(1/2))

RMSE Per Column
error_open      4.313992e+00
error_high      5.488284e+00
error_low       5.390995e+00
error_close     7.471622e+00
error_volume    1.690907e+07
dtype: float64


In [19]:
def plotly_residual_analysis(df, title_add=''):
    """
    Perform residual analysis for multiple features in a DataFrame.
    The DataFrame should contain actual and predicted columns for each feature.
    
    Args:
        df (pd.DataFrame): DataFrame containing actual and predicted columns.
        title_add (str, optional): Additional title for the subplots.
    """
    # Number of columns
    columns = [col for col in df.columns if not col.startswith('pred_')]
    num_features = len(columns)

    # Color per column
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'yellow']

    # Subplots per columns
    fig = make_subplots(rows=num_features, cols=4, vertical_spacing=0.035, horizontal_spacing=0.035,
                        subplot_titles=("Histogram", "QQ-Normal Plot", "Residuals vs. Predicted Values", "Residuals vs Index"))

    for i, col in enumerate(columns):
        actual = df[col]
        predicted = df[f'pred_{col}']
        residuals = actual - predicted
        mean_residuals = np.mean(residuals)
        sd_residuals = np.std(residuals)
        rmse = np.sqrt(np.mean(residuals**2))
        index = df.index

        # Assign color for each feature
        color = colors[i % len(colors)]

        # Histogram of residuals
        fig.add_trace(go.Histogram(x=residuals, nbinsx=30, name=f'{col.title()} Residuals', marker_color=color), row=i+1, col=1)
        # Add lines for mean and standard deviation
        fig.add_vline(x=mean_residuals, line=dict(color='black', width=2), row=i+1, col=1)
        fig.add_vline(x=mean_residuals + sd_residuals, line=dict(color='grey', width=2, dash='dash'), row=i+1, col=1)
        fig.add_vline(x=mean_residuals - sd_residuals, line=dict(color='grey', width=2, dash='dash'), row=i+1, col=1)
        fig.add_annotation(x=mean_residuals, y=5, text=f"Mean: {mean_residuals:.2f}", showarrow=True, row=i+1, col=1)
        fig.add_annotation(x=sd_residuals + mean_residuals, y=5, text=f"SD: {sd_residuals:.2f}", showarrow=False, row=i+1, col=1)
        
        # QQ-Normal of residuals
        qq = stats.probplot(residuals, dist="norm", plot=None)
        fig.add_trace(go.Scatter(x=qq[0][0], y=qq[1][1] + qq[1][0]*qq[0][0], mode='lines',  showlegend=False), row=i+1, col=2)
        fig.add_trace(go.Scatter(x=qq[0][0], y=qq[0][1], mode='markers', marker_color=color, name=f'{col.title()} QQ'), row=i+1, col=2)

        # Residuals vs. predicted values
        fig.add_trace(go.Scatter(x=predicted, y=residuals, mode='markers', marker_color=color, name=f'{col.title()} Resid Pred'), row=i+1, col=3)
        fig.add_hline(y=0, line=dict(color='red'), row=i+1, col=3)
        fig.add_hline(y=2 * rmse, line=dict(color='red', dash='dash'), row=i+1, col=3)
        fig.add_hline(y=-2 * rmse, line=dict(color='red', dash='dash'), row=i+1, col=3)

        # Residuals vs. index
        fig.add_trace(go.Scatter(x=index, y=residuals, mode='markers', marker_color=color, name=f'{col.title()} Resid Index'), row=i+1, col=4)
        fig.add_hline(y=0, line=dict(color='red'), row=i+1, col=4)
        fig.add_hline(y=2 * rmse, line=dict(color='red', dash='dash'), row=i+1, col=4)
        fig.add_hline(y=-2 * rmse, line=dict(color='red', dash='dash'), row=i+1, col=4)

    # Update layout
    fig.update_layout(height=250*num_features, width=1400, title_text="Residual Analysis " + title_add)
    fig.show()

In [20]:
# Residual Analysis Plot
plotly_residual_analysis(df_test_pred, 
                         title_add=f'- {SYMBOL} Vanilla LSTM')

<a name='pred'>

## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">5.3| LSTM Predictions </p>

In [21]:
def plot_predictions(y_values_df, predictions_df, title_add=''):
    """
    Plots actual values and predictions for each feature in separate subplots.
    
    Args:
        y_values_df (pd.DataFrame): DataFrame containing actual values.
        predictions_df (pd.DataFrame): DataFrame containing predicted values.
        title_add (str, optional): Additional title for the subplots.
    """
    # Number/color per features 
    columns = [col for col in y_values_df.columns]
    num_features = len(columns)
    actual_colors = ['cyan', 'lime', 'yellow', 'violet', 'gold', 'pink']

    # Creating subplots
    fig = make_subplots(rows=num_features, cols=1, vertical_spacing=0.03, subplot_titles=[col.title() for col in columns])

    for i, col in enumerate(columns):
        # Actual values trace
        fig.add_trace(go.Scatter(x=y_values_df.index, y=y_values_df[col], mode='lines', name=col.title(),
                                 line=dict(color=actual_colors[i % len(actual_colors)])), row=i+1, col=1)
        
        # Predicted values trace
        pred_col = f'pred_{col}'
        if pred_col in predictions_df.columns:
            fig.add_trace(go.Scatter(x=predictions_df.index, y=predictions_df[pred_col], 
                                     mode='lines', name=f'Predicted {col.title()}', line=dict(color='red')), row=i+1, col=1)
            
            # Calculate RMSE and add as an annotation
            rmse = np.sqrt(np.mean((y_values_df[col] - predictions_df[pred_col]) ** 2))
            fig.add_annotation(xref='x domain', yref='y domain', x=1, y=0.05, showarrow=False,
                               text=f'RMSE: {rmse:.2f}', row=i+1, col=1, font=dict(color='red'))
    fig.update_layout(height=350*num_features, width=1100, title_text="Data & Predictions " + title_add)
    fig.show()


In [22]:
# Plotting prediction and data
plot_predictions(df_y_test, df_predictions, title_add=f'- {SYMBOL} Vanilla LSTM')

### <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red;">5.3.1| Forecasting with LSTM Model </p>

In [23]:
# Getting last steps for LSTM forecast
last_steps = df[columns].iloc[-mapping_steps:, :]

# Number of steps to forecast
n_forecast_steps = 10 # ~ 2 weeks in business days

# Forming date index
date_index = pd.date_range(start=end_date, periods=n_forecast_steps, freq=BDay())

# Forecasting n-steps
forecast_array = rnn_formater.forecast_n_steps(model, last_steps, n_forecast_steps)
forecast = pd.DataFrame(forecast_array, index=date_index, columns=[f'forecast_{col}' for col in columns])
show(forecast)

,forecast_open,forecast_high,forecast_low,forecast_close,forecast_volume
2023-12-01 00:00:00,239.591,244.078,234.510,240.567,127801463.936
2023-12-04 00:00:00,239.800,244.571,234.786,240.992,126029865.472
2023-12-05 00:00:00,240.104,244.900,235.110,241.312,124952504.226
2023-12-06 00:00:00,240.265,245.029,235.259,241.425,124334107.440
2023-12-07 00:00:00,240.286,245.016,235.265,241.394,123941247.942
2023-12-08 00:00:00,240.204,244.911,235.173,241.272,123653270.500
2023-12-11 00:00:00,240.056,244.748,235.020,241.095,123410621.986
2023-12-12 00:00:00,239.867,244.550,234.832,240.887,123189731.622
2023-12-13 00:00:00,239.655,244.332,234.625,240.664,122983058.866
2023-12-14 00:00:00,239.432,244.105,234.408,240.433,122788013.307


In [24]:
# Plotting function
def plot_stock_data(df, previous_data=None, test_data=None, title_add=''):
    colors = ['blue', 'green', 'purple', 'orange', 'cyan']

    fig = make_subplots(rows=df.shape[1], cols=1, shared_xaxes=True, vertical_spacing=0.02,
                        subplot_titles=df.columns)

    for i, col in enumerate(df.columns):
        fig.add_trace(
            go.Scatter(x=df.index, y=df[col], mode='lines', name=col, line=dict(dash='dashdot', color='red'),
                      ), row=i+1, col=1)

        if previous_data is not None:
            column = list(previous_data.columns)[i]
            fig.add_trace(go.Scatter(x=previous_data.index, y=previous_data[column], mode='lines', name=column,
                                     line=dict(color=colors[i % len(colors)])), row=i+1, col=1)

        if test_data is not None:
            column = list(test_data.columns)[i]
            fig.add_trace(
                go.Scatter(x=test_data.index, y=test_data[column], mode='lines', name=f'Unseen {col.title()}',
                           line=dict(color='floralwhite')), row=i+1, col=1)
            

    fig.update_layout(height=1200, width=1100, title_text=f"Stock Data Over Time {title_add}")
    fig.show()

In [25]:
# Actual stock values to test forecast
df_test = ticker.history(start=end_date, end='2023-12-15').iloc[-n_forecast_steps:, :]
df_test.columns = df_test.columns.str.lower()

# Plotting forecast
plot_stock_data(forecast, last_steps, df_test, title_add=f'- {SYMBOL} Vanilla LSTM')

In [26]:
# Error per column on New Data
error_array = forecast.values - df_test[columns].values
errors = pd.DataFrame(error_array, index=df_test.index, columns=[f'error_{col}' for col in columns])
show(errors)

,error_open,error_high,error_low,error_close,error_volume
Date,,,,,
2023-12-01 00:00:00-05:00,6.451,3.888,2.610,1.737,6627963.936
2023-12-04 00:00:00-05:00,4.050,5.201,1.496,5.412,21930065.472
2023-12-05 00:00:00-05:00,6.234,-1.760,1.410,2.592,-13018595.774
2023-12-06 00:00:00-05:00,-2.655,-1.541,-3.911,2.055,-2102092.560
2023-12-07 00:00:00-05:00,-1.264,0.936,-1.715,-1.246,16798947.942
2023-12-08 00:00:00-05:00,-0.066,-0.359,-4.097,-2.568,20673170.500
2023-12-11 00:00:00-05:00,-2.684,1.308,-2.430,1.355,25496721.986
2023-12-12 00:00:00-05:00,1.317,5.560,0.962,3.877,27861431.622
2023-12-13 00:00:00-05:00,5.465,4.032,6.425,1.374,-23303241.134


In [27]:
# RMSE per column on New Data
root_mean_squared_error = np.sqrt((errors**2).mean())
root_mean_squared_error = pd.DataFrame(root_mean_squared_error, columns=['RMSE New TLSA Data'])
show(root_mean_squared_error.T)

,error_open,error_high,error_low,error_close,error_volume
RMSE New TLSA Data,3.841,4.394,3.673,4.276,21965564.125


- This is very simimilar to the error on the test data used in the validation of the LSTM above!

<a name='mlstm'>


# <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -65px;">6| Multi-Step Stacked LSTM Model </p>
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">6.1| Multi-Step Mathematical Background </p>

在使用lstm的多步时间序列预测场景中，输入序列到输出序列的映射是一个关键方面。这个过程包括使用历史数据序列来预测未来数据点的序列。

＃＃＃＃多步骤映射的一般表示
1. **输入序列(X):**给定一个时间序列，LSTM模型的输入是在固定数量的过去时间步长上收集的数据点序列。对于具有' n '个时间步长的序列，输入序列$ X_i $表示为:
$$ X_i = [x_{i}, x_{i+1}, ..., x_{i+n-1}] $$
其中$ x_j $表示时间$ j $的数据点。

2. **输出序列(y):** LSTM模型的任务是预测未来特定步数的值。为了预测未来的“m”步数，模型的输出序列$ y_i $为:
$$ y_i = [y_{i+n}, y_{i+n+1}, ..., y_{i+n+m-1}] $$
其中$ y_k $是时间$ k $的预测值。

＃＃＃＃序列映射的数学公式
我们将$ t $表示为当前时间步长，LSTM模型处理每个输入序列以产生相应的输出序列。该映射可以抽象地表示为:
$$\textbf{LSTM}(X_i)\rightarrow y_i $$

这意味着LSTM接受输入序列$ X_i $并产生输出序列$ y_i $，这实际上是对下一个“m”时间步长的预测。这种方法允许LSTM分析一段时间内的模式，并对即将到来的时间框架的股票价格趋势做出预测。 

**关键提示:**多步预测的核心挑战是有效地学习和利用更长的历史时期(输入序列)的时间关系来预测未来序列。这比单步预测更复杂，单步预测中模型只预测下一个值。然而，其优点是能够提供更全面和更长期的预测。

<a name='background'>
    
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">6.2| Understanding Stacked LSTM </p>
   
在时间序列预测中，例如预测股票价格，我们经常使用长短期记忆(LSTM)网络，因为它们能够捕获时间依赖性。LSTM模型的两种常见架构是单层LSTM和堆叠(多层)LSTM。每一个都有适合于时间序列数据中不同复杂性的独特特征。

＃＃＃＃单层LSTM
单层LSTM由一个LSTM层组成。它相对简单，可以有效地为时间序列数据建模，其中时间步之间的关系并不过于复杂。这种体系结构对于较短的序列或较少易失性的数据特别有效。

- **简单性:**更容易训练，更不容易在较小的数据集上过度拟合。
- **速度:**通常更快的训练由于较少的参数。
**适用于更直接的时间序列问题，其中过去和未来数据点之间的关系更线性或波动性更小。
- **数学过程:**每个时间步长的输入都通过这个单层内的一系列门(遗忘，输入和输出)进行处理，这影响隐藏状态和细胞状态。
- **方程式:** LSTM单元的更新是基于当前输入和之前的隐藏状态。数学运算被限制在单一的一系列变换中:
$$ h_t, C_t = LSTM(x_t, h_{t-1}, C_{t-1}) $$
其中$ h_t $和$ C_t $是时间$ t $的隐藏状态和单元状态，$ x_t $是时间$ t $的输入状态。


＃＃＃＃堆叠LSTM
相比之下，堆叠LSTM涉及多个LSTM层相互堆叠。每一层都为下一层提供信息，使模型能够从更细微的数据表示中学习。

- **复杂性:**更善于捕捉数据中的复杂模式和关系。每一层都可以了解数据的不同方面，提供更详细的分析。
- **灵活性:**可以建模更复杂和更长的序列，使其适用于波动或非线性时间序列数据，如股票价格。
- **深度:**额外的层提供了更深入的理解，使模型能够捕捉更微妙的时间动态。
- **数学过程:**第一层LSTM处理输入序列，并将其隐藏状态输出传递给第二层。这个过程在所有层中重复，每层学习数据的不同方面。
- **方程式:**转换发生顺序跨层。对于两层堆叠的LSTM，过程可以描述为:
$$ h_t^{(1)}, C_t^{(1)} = LSTM^{(1)}(x_t, h_{t-1}^{(1)}, C_{t-1}^{(1)}) $$
$$ h_t^{(2)}, C_t^{(2)} = LSTM^{(2)}(h_t^{(1)}, h_{t-1}^{(2)}, C_{t-1}^{(2)}) $$
上标表示层数。

  
＃＃＃＃
- **性能:**堆叠lstm往往在复杂问题上表现更好，但它们需要更多的数据和计算资源。单层lstm在简单的任务上效率更高。
- **应用:**选择单层lstm还是堆叠lstm取决于时间序列问题的具体要求。例如，股票市场预测以其波动性和非线性而闻名，通常受益于堆叠lstm提供的深度。
- **学习深度:**在堆叠lstm中，数据经过多层处理，使模型能够捕获时间序列中更复杂和抽象的特征。
- **容量:**由于额外的层，堆叠lstm具有更高的建模复杂问题的能力，但需要更多的数据来有效地训练而不会过度拟合。
- **计算复杂度:**由于参数和层数的增加，堆叠lstm的计算强度更高[1,2]。

<a name='mutlidataprep'>
    
## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">6.3| Mutli-Step LSTM Data Preperation </p>

In [28]:
class RNNFormaterMultiStep:
    def __init__(self, data: pd.DataFrame, n_steps_in, n_steps_out):
        """
        Initialize the RNNFormater with a DataFrame, number of input steps, and number of output steps.
        
        Args:
            data (pd.DataFrame): Input DataFrame containing time series data.
            n_steps_in (int): Number of time steps for each input sequence.
            n_steps_out (int): Number of time steps for each output sequence.
        """
        self.df = data.copy()
        self.n_steps_in = n_steps_in
        self.n_steps_out = n_steps_out
        self.n_columns = self.df.shape[1]
        
        self.scaler = MinMaxScaler()
        self.normalized_data = self.scaler.fit_transform(self.df.values)

    def data_mapping(self):
        """
        Maps a 2D array into a 3D array for RNN input, with each sequence having n_steps_in time steps
        and each target having n_steps_out time steps.
        
        Returns:
            X (np.array): A 3D array of input sequences.
            y (np.array): A 3D array of target sequences.
        """
        num_samples = len(self.normalized_data) - self.n_steps_in - self.n_steps_out + 1

        X = np.empty((num_samples, self.n_steps_in, self.n_columns))
        y = np.empty((num_samples, self.n_steps_out, self.n_columns))

        for i in range(num_samples):
            X[i, :, :] = self.normalized_data[i:i + self.n_steps_in, :]
            y[i, :, :] = self.normalized_data[i + self.n_steps_in:i + self.n_steps_in + self.n_steps_out, :]

        return X, y

    def rnn_train_test_split(self, X, y, test_percent=0.1):
        """
        Splits the 3D mapped data into training and testing sets for an RNN.
        
        Args:
            X (np.array): The input data sequences.
            y (np.array): The target data sequences.
            test_percent (float): The fraction of data to be used for testing.
        
        Returns:
            X_train, X_test, y_train, y_test (tuple): Split data into training and testing sets.
        """
        test_size = int(len(X) * test_percent)
        X_train, y_train = X[:-test_size], y[:-test_size]
        X_test, y_test = X[-test_size:], y[-test_size:]

        return X_train, X_test, y_train, y_test

    def multi_step_forecast(self, model, data: np.array):
        """
        Forecast multiple steps ahead using the LSTM model.
        
        Args:
            model (tf.keras.Model): Trained LSTM model for prediction.
            data (pd.DataFrame): Input DataFrame containing the latest time series data.
        
        Returns:
            pd.DataFrame: Forecasted values for n_steps_out steps.
        """    
        # Normalizing latest data
        last_steps_normalized = self.scaler.transform(data)
        last_steps_normalized = last_steps_normalized.reshape(1, self.n_steps_in, self.n_columns)
        
        # Predicting using model
        forecast = model.predict(last_steps_normalized)
        forecast = forecast.reshape(self.n_steps_out, self.n_columns)

        # Inverse transforming to original scale
        forecast = self.scaler.inverse_transform(forecast)
        forecast = pd.DataFrame(forecast, columns=[f'forecast_{col}' for col in self.df.columns])
        
        return forecast
        

<a name='buildmlstm'>

## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">6.4| Building Stacked Mutli-Step LSTM </p>
    
这里，我们再次使用来自sklearn的“MinMaxScaler”。预处理'，这在类' RNNFormaterMultiStep '中很好地完成了
​
在我们的例子中，我们有我们的特斯拉股价预测模型:**
- **输入(X):**我们映射过去股票价格的“252”时间步，代表大约“1年”的股票数据在工作日。
- **输出(y):**模型预测下一个“21”时间步骤，相当于“1个月”提前工作日。
​
---
​
**是什么**:在神经网络中，例如在Keras/TensorFlow中，' kernel_initializer '是一个参数，用于设置初始化层中权重的方法。
​
* * * *:数学基础
—正交初始化:该方法将权重设置为正交矩阵，保留了层从输入到输出的方差。在数学上，对于一个权重矩阵$ W $，它保证$ W^T W = I $或$ W W^T = I $，其中$ I $是单位矩阵。
​
**为什么重要**:
-正确的权重初始化对有效训练至关重要。它会影响网络的收敛速度和最终性能。
正交初始化在深度网络中是有益的，因为它避免了梯度的消失和爆炸。
​

### <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px">6.4.1| Bidirectional LSTM </p>
    
双向LSTM是一种用于处理和分析序列数据的RNN架构。与传统lstm在一个方向上读取序列不同，双向lstm同时在正向和反向处理数据。这种双向处理允许模型从过去和未来的上下文中捕获依赖关系。

在训练阶段，双向LSTM考虑从前一个时间步到后一个时间步的信息，反之亦然。这有助于模型理解可能存在于序列中不同点的模式。双向LSTM由两个独立的层组成——一个层从开始到结束处理序列，另一个层从结束到开始处理序列。来自两个方向的输出通常被连接或组合，以提供更全面的输入序列表示。

这种架构在需要同时理解过去和未来信息的任务中非常有用，比如时间序列分析和自然语言处理。通过合并双向处理，LSTM可以更熟练地捕获顺序数据中的复杂关系。

**双向模型示例**

```python
# Multi-Step LSTM
ms_model = models.Sequential([
    LSTM(units=40, return_sequences=True, input_shape=(n_steps_in, len(columns)), kernel_initializer='orthogonal'),
    Bidirectional(LSTM(units=20, return_sequences=False)),
    Dense(units=n_steps_out * len(columns))   
])
```

In [29]:
n_steps_in = 252  # ~ 1 year in business days mapping
n_steps_out = 21  # ~ 1 month forecast in business days

# Intializing created RNN class
ms_rnn_formatter = RNNFormaterMultiStep(df[columns], n_steps_in, n_steps_out)

# Normalizing and data mappings for input & output
X, y = ms_rnn_formatter.data_mapping()

# Train Test split
X_train, X_test, y_train, y_test = ms_rnn_formatter.rnn_train_test_split(X, y)

print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}') # (time mapes, time steps, columns)
print(f'\nX_test shape: {X_test.shape}\ny_test shape: {y_test.shape}')

X_train shape: (1322, 252, 5)
y_train shape: (1322, 21, 5)

X_test shape: (146, 252, 5)
y_test shape: (146, 21, 5)


### <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red;">6.4.2 | Training Stacked Mutli-Step LSTM </p>

In [30]:
# For consistant results
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# Multi-Step LSTM
ms_model = models.Sequential([
    
    # Layer 1
    LSTM(units=40, return_sequences=True, input_shape=(n_steps_in, len(columns)), kernel_initializer='orthogonal'),

    # Layer 2
    LSTM(units=20, return_sequences=False),
    # Bidirectional(LSTM(units=20, return_sequences=False)),

    # Output layer
    Dense(units=n_steps_out * len(columns))   
])

ms_model.compile(optimizer='adam', loss='mse')
ms_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 252, 40)           7360      
                                                                 
 lstm_2 (LSTM)               (None, 20)                4880      
                                                                 
 dense_1 (Dense)             (None, 105)               2205      
                                                                 
Total params: 14445 (56.43 KB)
Trainable params: 14445 (56.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Callback to save model weights
model_checkpoint = ModelCheckpoint('Multi_LSTM_Tesla_model.h5', monitor='val_loss', save_best_only=True)

# Fitting the model
ms_history = ms_model.fit(X_train, y_train.reshape(-1, n_steps_out * len(columns)), 
                          batch_size=1024,
                          epochs=150, 
                          validation_data=(X_test, y_test.reshape(-1, n_steps_out * len(columns))), 
                          callbacks=[model_checkpoint],
                          shuffle=False, 
                          verbose=False)

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [32]:
# Plotting stakced multi-step model loss
plot_training_history(ms_history.history,
                      plot_title='Stacked Multi-Step LSTM Model Loss')

<a name='forecast'>

## <p style="background-color: red; font-family: Tahoma; font-size:150%; color: white; text-align: left; border-radius: 5px; padding:7px; border:solid 5px red; margin-bottom: -10px;">6.5| Mutli-Step Forecasting with Stacked Multi-Step LSTM </p>

In [33]:
# Getting stock data for forecasting
data_to_test = df[columns].iloc[-n_steps_in:]

# Showing data
print(f'Test Data Shape: {data_to_test.shape}')
show(data_to_test.tail())

Test Data Shape: (252, 5)


,open,high,low,close,volume
Date,,,,,
2023-11-24 00:00:00-05:00,233.750,238.750,232.330,235.450,65125200
2023-11-27 00:00:00-05:00,236.890,238.330,232.100,236.080,112031800
2023-11-28 00:00:00-05:00,236.680,247.000,234.010,246.720,148549900
2023-11-29 00:00:00-05:00,249.210,252.750,242.760,244.140,135401300
2023-11-30 00:00:00-05:00,245.140,245.220,236.910,240.080,132353200


In [34]:
# Loading best wieghts during training
ms_model = models.load_model('Multi_LSTM_Tesla_model.h5')

# Forecasting using rnn
forecast = ms_rnn_formatter.multi_step_forecast(ms_model, data_to_test.values)

# Setting index as datetime corresponding to forecast period
forecast.index = pd.date_range(start=end_date, periods=n_steps_out, freq=BDay())
print(f'Forecast shape: {forecast.shape}')
show(forecast)

1/1 [==============================] - 1s 825ms/step
Forecast shape: (21, 5)


,forecast_open,forecast_high,forecast_low,forecast_close,forecast_volume
2023-12-01 00:00:00,249.410,260.705,238.104,239.419,84643960.000
2023-12-04 00:00:00,243.813,253.799,245.789,236.021,97944256.000
2023-12-05 00:00:00,254.510,256.411,246.771,231.873,99097400.000
2023-12-06 00:00:00,259.885,263.816,242.554,251.097,120292216.000
2023-12-07 00:00:00,239.011,254.609,244.017,252.881,87769224.000
2023-12-08 00:00:00,243.505,258.082,244.732,254.794,83395552.000
2023-12-11 00:00:00,249.511,261.650,231.726,241.131,99479704.000
2023-12-12 00:00:00,247.244,258.670,224.309,247.477,114101344.000
2023-12-13 00:00:00,250.577,260.931,235.671,248.537,110833744.000
2023-12-14 00:00:00,241.886,254.810,227.824,248.612,99570872.000


In [35]:
def plot_forecast(previous_data, forecast, test_data=None, title_add=''):
    """
    Plots actual values and predictions for each feature in separate subplots.
    
    Args:
        previous_data (pd.DataFrame): DataFrame containing actual values.
        forecast (pd.DataFrame): DataFrame containing predicted values.
        title_add (str, optional): Additional title for the subplots.
    """
    # Number/color per features 
    columns = [col for col in previous_data.columns]
    num_features = len(columns)
    actual_colors = ['cyan', 'gold', 'violet', 'lime', 'blue', 'pink', 'yellow']

    # Creating subplots
    fig = make_subplots(rows=num_features, cols=1, vertical_spacing=0.03, subplot_titles=[col.title() for col in columns])

    for i, col in enumerate(columns):
        # Actual values trace
        fig.add_trace(
            go.Scatter(x=previous_data.index, y=previous_data[col], mode='lines', name=col.title(),
                       line=dict(color=actual_colors[i % len(actual_colors)])), row=i+1, col=1)
        
        # Predicted values trace
        pred_col = f'forecast_{col}'
        if pred_col in forecast.columns:
            fig.add_trace(
                go.Scatter(x=forecast.index, y=forecast[pred_col], 
                           mode='lines', name=f'Forecast {col.title()}', line=dict(color='red')), row=i+1, col=1)

        if test_data is not None:
            fig.add_trace(
                go.Scatter(x=test_data.index, y=test_data[col], mode='lines', name=f'Unseen {col.title()}',
                           line=dict(color='floralwhite')), row=i+1, col=1)
            
            
    fig.update_layout(height=350*num_features, width=1100, title_text="Data and Forecast " + title_add)
    fig.show()


In [36]:
# Actual stock values to test forecast
df_test = ticker.history(start=end_date).iloc[-n_steps_out:, :]
df_test.columns = df_test.columns.str.lower()

# Plotting stacked mutli-step LSTM forecast
plot_forecast(data_to_test[columns], forecast, df_test, title_add='- TSLA Stock')



- [1]. Mayank, M. (2020, October 17). A practical guide to RNN and LSTM in Keras. Medium. https://towardsdatascience.com/a-practical-guide-to-rnn-and-lstm-in-keras-980f176271bc 
- [2]. Understanding LSTM networks. Understanding LSTM Networks -- colah’s blog. (n.d.). https://colah.github.io/posts/2015-08-Understanding-LSTMs/ 

---